# LeftJoin_Meteo_Estacion_1.1

En este script vamos a tratar de conseguir lo que no hicimos en la version 1.0, a saber, tener en un mismo fichero la relacion de estaciones-meteo-coordenadas.

Para alcanzar este objetivo primero hemos acudido al codigo fuente de la pagina web de la que obtenemos los datos de meteo, y hemos conseguido un listado de las estaciones con su id asociado, montando un fichero csv (estaciones_codigo_fuente_def.csv).

Posteriormente hemos montado un fichero que contiene ID, Estacion y sus coordenadas (estaciones_coordenadas_2.csv).

Una vez hecho esto, desarrollamos el script para relacionar estaciones-meteo-coordenadas (historico_estaciones_coordenadas_2.csv).


In [1]:
#Leemos los ficheros: 
#en el primero tenemos la relacion ID-ESTACION de la web
#en el segundo el listado de estaciones con sus coordenadas, obtenido del dropbox
import numpy as np
import pandas as pd

df_est_cf=pd.read_csv('../DATASETS/ESTACIONES/estaciones_codigo_fuente_def.csv',sep='|',error_bad_lines=False)
df_estaciones = pd.read_csv('../DATASETS/ESTACIONES/OMM_Estaciones.csv',sep='|',error_bad_lines=False)

In [2]:
#Analizamos el numero de campos, registras, asi como su tipo
df_est_cf.head()

,ID,ESTACION
0,8427,ABLA
1,8486,"ADRA,CLUB NAUTICO"
2,8432,AGUILAS
3,8019,AGUION
4,8380,"ALAJAR, CABEZO MOLIN"


In [3]:
df_estaciones.head()

,IIiii,OACI,Nombre,Estado,Latitud,Longitud,Altitud,Notas
0,8427.0,NaN,ABLA,SPAIN,37-08-29N,002-46-46W,885,NaN
1,NaN,LETU,"ABLITAS, AERODROMO",SPAIN,42-00-40N,001-37-26W,324,NaN
2,8486.0,NaN,ADRA - CLUB NAUTICO,SPAIN,36-44-48N,003-01-14W,8,NaN
3,8432.0,NaN,AGUILAS,SPAIN,37-25-00N,001-35-12W,32,NaN
4,8019.0,NaN,AGUION,SPAIN,43-28N,006-18W,923,NaN


In [4]:
#left join para añadir al dataset de id-estacion las coordenadas de las estaciones

result = pd.merge(df_est_cf, 
                  df_estaciones[['IIiii','Latitud','Longitud']],
                  left_on='ID',
                  right_on='IIiii',
                  how='left')

In [5]:
result.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 269 entries, 0 to 268
Data columns (total 5 columns):
ID          269 non-null int64
ESTACION    269 non-null object
IIiii       269 non-null float64
Latitud     269 non-null object
Longitud    269 non-null object
dtypes: float64(1), int64(1), object(3)
memory usage: 12.6+ KB


In [6]:
result=result[['ID','ESTACION','Latitud','Longitud']]

In [7]:
result.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 269 entries, 0 to 268
Data columns (total 4 columns):
ID          269 non-null int64
ESTACION    269 non-null object
Latitud     269 non-null object
Longitud    269 non-null object
dtypes: int64(1), object(3)
memory usage: 10.5+ KB


In [8]:
result.to_csv('../DATASETS/ESTACIONES/estaciones_coordenadas.csv',sep='|',header=True,index=False)

In [9]:
pd.concat(g for _, g in df_estaciones.groupby("IIiii") if len(g) > 1)

,IIiii,OACI,Nombre,Estado,Latitud,Longitud,Altitud,Notas
50,8165.0,NaN,BUJARALOZ,SPAIN,41-31-18N,000-10-27W,357,Hasta 20180611
71,8165.0,NaN,CASPER,SPAIN,41-14-32N,000-04-06W,203,Desde 20180612
65,8936.0,NaN,CAP DE VAQUEIRA,SPAIN,42-41-31N,000-58-26E,2467,Desde 20141101
271,8936.0,NaN,SORT,SPAIN,42-25N,001-08E,680,Hasta 20141101
64,8940.0,NaN,CAP DE RAC,SPAIN,42-25-49N,001-40-04E,1940,Desde 20141101
279,8940.0,NaN,TAVESCAN,SPAIN,42-38N,001-15E,1100,Hasta 20141101


se puede observar que tenemos duplicados en 3 estaciones. esto se debe a que la estacion asociada al id, en cada caso, ha tenido dos localizaciones. lo que haremos sera eliminar estos duplicados del fichero "OMM_Estaciones.csv" y nos quedaremos con la que aparece en el codigo fuente de la web, y que a su vez es la que está vigente. El nuevo csv sin duplicados lo llamaremos OMM_Estaciones_NoDupli.csv

In [10]:
df_NoDupli=df_estaciones.sort_values(by=['Notas'])
#df_NoDupli.head(n=15)

df_NoDupli=df_NoDupli.drop_duplicates(['IIiii'], keep='first')
df_NoDupli.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 304 entries, 286 to 322
Data columns (total 8 columns):
IIiii       303 non-null float64
OACI        52 non-null object
Nombre      304 non-null object
Estado      304 non-null object
Latitud     304 non-null object
Longitud    304 non-null object
Altitud     304 non-null int64
Notas       6 non-null object
dtypes: float64(1), int64(1), object(6)
memory usage: 21.4+ KB


In [11]:
df_NoDupli.to_csv('../DATASETS/ESTACIONES/OMM_Estaciones_NoDupli.csv',sep='|',header=True,index=False)

In [12]:
#left join para añadir al dataset de meteo las coordenadas de las estaciones

result_2 = pd.merge(df_est_cf, 
                  df_NoDupli,
                  left_on='ID',
                  right_on='IIiii',
                  how='left')

In [13]:
result_2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 266 entries, 0 to 265
Data columns (total 10 columns):
ID          266 non-null int64
ESTACION    266 non-null object
IIiii       266 non-null float64
OACI        42 non-null object
Nombre      266 non-null object
Estado      266 non-null object
Latitud     266 non-null object
Longitud    266 non-null object
Altitud     266 non-null int64
Notas       5 non-null object
dtypes: float64(1), int64(2), object(7)
memory usage: 22.9+ KB


In [14]:
result_2=result_2[['ID','ESTACION','Estado','Latitud','Longitud']]

In [15]:
result_2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 266 entries, 0 to 265
Data columns (total 5 columns):
ID          266 non-null int64
ESTACION    266 non-null object
Estado      266 non-null object
Latitud     266 non-null object
Longitud    266 non-null object
dtypes: int64(1), object(4)
memory usage: 12.5+ KB


In [16]:
result_2.to_csv('../DATASETS/ESTACIONES/estaciones_coordenadas_2.csv',sep='|',header=True,index=False)

Con esto hemos conseguido el objetivo de relacionar estacion-coordenadas. A continuacion incluimos en la ecuacion los datos de meteo:

In [17]:
df_meteo=pd.read_csv('../DATASETS/ESTACIONES/historico_estaciones.csv',sep='|',error_bad_lines=False)

In [18]:
df_meteo['Estacion'].replace(':','', inplace=True, regex=True)

In [19]:
df_meteo.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 359612 entries, 0 to 359611
Data columns (total 12 columns):
Estacion            359612 non-null object
Fecha               359612 non-null object
T. med. (ºC)        359612 non-null object
T. max (ºC)         359612 non-null object
T. min (ºC)         359612 non-null object
Prec. (mm)          359612 non-null object
Presion/ Geopot.    359612 non-null object
Dir. vi.            359612 non-null object
Vel. vi. (Km/h)     359612 non-null object
Nub.                359612 non-null object
Prof. nieve (cm)    359612 non-null object
Insolac. (horas)    283707 non-null object
dtypes: object(12)
memory usage: 32.9+ MB


In [20]:
df_meteo.head()

,Estacion,Fecha,T. med. (ºC),T. max (ºC),T. min (ºC),Prec. (mm),Presion/ Geopot.,Dir. vi.,Vel. vi. (Km/h),Nub.,Prof. nieve (cm),Insolac. (horas)
0,LA CORUNA,31/01/2011,8.3,10.4,6.2,0.0,1024.1 Hpa,65º(NE),14,7/8,-,0.7
1,LUGO/ROZAS,31/01/2011,2.6,4.4,0.9,0.2,1024.3 Hpa,325º(NW),2,8/8,-,0.0
2,ASTURIAS/AVILES,31/01/2011,5.5,8.0,2.9,0.0,1025.0 Hpa,203º(SW),6,7/8,-,0.8
3,GIJON-MUSEL,31/01/2011,6.6,9.0,4.2,0.0,1024.1 Hpa,224º(SW),5,7/8,-,0.4
4,OVIEDO,31/01/2011,4.4,7.4,1.4,Ip,1024.5 Hpa,235º(SW),2,7/8,-,1.6


In [21]:
result_2.head()

,ID,ESTACION,Estado,Latitud,Longitud
0,8427,ABLA,SPAIN,37-08-29N,002-46-46W
1,8486,"ADRA,CLUB NAUTICO",SPAIN,36-44-48N,003-01-14W
2,8432,AGUILAS,SPAIN,37-25-00N,001-35-12W
3,8019,AGUION,SPAIN,43-28N,006-18W
4,8380,"ALAJAR, CABEZO MOLIN",SPAIN,37-52-05N,006-40-27W


In [22]:
result_3 = pd.merge(df_meteo, 
                  result_2,
                  left_on='Estacion',
                  right_on='ESTACION',
                  how='left')

In [23]:
result_3.head()

,Estacion,Fecha,T. med. (ºC),T. max (ºC),T. min (ºC),Prec. (mm),Presion/ Geopot.,Dir. vi.,Vel. vi. (Km/h),Nub.,Prof. nieve (cm),Insolac. (horas),ID,ESTACION,Estado,Latitud,Longitud
0,LA CORUNA,31/01/2011,8.3,10.4,6.2,0.0,1024.1 Hpa,65º(NE),14,7/8,-,0.7,8001,LA CORUNA,SPAIN,43-22N,008-25W
1,LUGO/ROZAS,31/01/2011,2.6,4.4,0.9,0.2,1024.3 Hpa,325º(NW),2,8/8,-,0.0,8008,LUGO/ROZAS,SPAIN,43-07N,007-27W
2,ASTURIAS/AVILES,31/01/2011,5.5,8.0,2.9,0.0,1025.0 Hpa,203º(SW),6,7/8,-,0.8,8011,ASTURIAS/AVILES,SPAIN,43-33N,006-02W
3,GIJON-MUSEL,31/01/2011,6.6,9.0,4.2,0.0,1024.1 Hpa,224º(SW),5,7/8,-,0.4,8014,GIJON-MUSEL,SPAIN,43-33-36N,005-42-03W
4,OVIEDO,31/01/2011,4.4,7.4,1.4,Ip,1024.5 Hpa,235º(SW),2,7/8,-,1.6,8015,OVIEDO,SPAIN,43-21N,005-52W


In [24]:
result_3=result_3[['Fecha','ID','Estacion','Estado','Latitud','Longitud','T. med. (ºC)','T. max (ºC)','T. min (ºC)','Prec. (mm)','Presion/ Geopot.','Dir. vi.','Vel. vi. (Km/h)','Nub.','Prof. nieve (cm)','Insolac. (horas)']]

In [25]:
result_3.to_csv('../DATASETS/ESTACIONES/historico_estaciones_coordenadas_2.csv',sep='|',header=True,index=False)

hay que modificar el fichero de estaciones_coordenadas_2.csv, ya que se estan incluyendo estaciones que no aparecen en el fichero de historico de meteo y a la postre hara que tengamos incidentes asociados a estaciones meteorologicas que no tienen datos meteorologicos.

In [26]:
df_estacion=result_3[['Estacion']]

In [27]:
df_estacion=pd.DataFrame(df_estacion.Estacion.unique(),columns=['Estacion'])


In [28]:
result_4 = pd.merge(df_estacion, 
                  result_2,
                  left_on='Estacion',
                  right_on='ESTACION',
                  how='left')

In [29]:
result_4=result_4[['ID','ESTACION','Estado','Latitud','Longitud']]

In [30]:
result_4.to_csv('../DATASETS/ESTACIONES/estaciones_coordenadas_2.csv',sep='|',header=True,index=False)

## Analisis 

Se ha conseguido condensar en un solo fichero el histórico de meteo en cada estacion, asi como las coordenadas del lugar donde esta instalada en cada caso.

## Conclusion

Una vez obtenido el fichero historico_estaciones_coordenadas_2.csv, tenemos que relacionar el lugar de ahogamiento en cada caso, con las condiciones meteorologicas del lugar en el que se produce el incidente.

Esto ultimo no es trivial, ya que las estaciones estan colocadas en lugares concretos, con unas coordenadas concretas, que no tienen por que ser las del lugar del ahogamiento. Habra que hacer algunas aproximaciones para poder relacionar ambas fuentes de informacion.

Por otro lado el formato en el que tenemos las coordenadas difiere en ambos ficheros. En uno las tenemos en formato decimal, y en el otro en sexagesimal.

## Propuesta 

Para la unificacion del formato de las coordenadas se plantean dos posibilidades:
    
    --Transformar "a huevo" las coordenadas

Para asociar una estacion meterologica a cada incidente se propone:
    
    --para cada incidente, recorrer el fichero de estaciones-coordenadas, calculando la distancia entre el incidente y y las       diferentes estaciones, y quedandonos con aquella cuya distancia sea menor.